In [ ]:
!pip install tensorflow opencv-python matplotlib

In [ ]:
!pip list

In [ ]:
import tensorflow as tf
import os

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
#!unzip "/content/drive/MyDrive/Lavanya K/images.zip"

In [ ]:
import cv2
import imghdr

In [ ]:
import pandas as pd

data = pd.read_csv("Dataset/Dataset.csv")
data.head()

In [ ]:
features = ["SEASON", "SUBBASIN", "ISO_TIME", "WMO_WIND"]
#features.append("WMO_PRES")
data = data[features]
data.head()

## Mapping Images to the Winddata

In [ ]:
from datetime import datetime

date_series = data["ISO_TIME"]
datetime_series = pd.to_datetime(date_series, format="%d-%m-%Y %H:%M")
formatted_series = datetime_series.dt.strftime("%d%b%Y_%H%M").str.upper()
file_name = "3DIMG_" + formatted_series + "_L1C_ASIA_MER_IR1_V01R00.jpg"

file_name.head()

In [ ]:
data["IR1"] = file_name

In [ ]:
data.head()

## Wind column Preprocessing

In [ ]:
data['WMO_WIND'] = data['WMO_WIND'].replace(' ', 0)
data['WMO_WIND'] = pd.to_numeric(data['WMO_WIND'], errors='coerce')

## Highlighting the clouds of the Images

In [ ]:
def image_highlighting(cropped_image):
    
    img_array = np.array(cropped_image)

    single_channel = img_array[:, :, 0]

    ret1, thresh = cv2.threshold(single_channel, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    kernel = np.ones((3, 3), np.uint8)
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)

    opening = clear_border(opening)
    sure_bg = cv2.dilate(opening, kernel, iterations=10)
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    ret2, sure_fg = cv2.threshold(dist_transform, 0.5 * dist_transform.max(), 255, 0)
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg, sure_fg)

    ret3, markers = cv2.connectedComponents(sure_fg)
    markers = markers + 10
    markers[unknown == 255] = 0

    markers = cv2.watershed(img_array, markers)

    img_array[markers == -1] = [255, 87, 51]
    img2 = color.label2rgb(markers, bg_label=0)
    plt.subplot(1, 2, 2)
    plt.imshow(img_array)
    plt.axis('off')
    plt.title('Watershed')

    plt.tight_layout()
    plt.show()
    
    return img_array

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

def image_highlighting_grabcut(cropped_image):
    img_array = np.array(cropped_image)

    # Initialize rectangle for GrabCut
    h, w = img_array.shape[:2]
    rect = (10, 10, w-20, h-20)

    mask = np.zeros(img_array.shape[:2], dtype=np.uint8)
    bgdModel = np.zeros((1, 65), dtype=np.float64)
    fgdModel = np.zeros((1, 65), dtype=np.float64)

    # Run GrabCut algorithm
    cv2.grabCut(img_array, mask, rect, bgdModel, fgdModel, 5, cv2.GC_INIT_WITH_RECT)

    # Create mask where sure foreground and likely foreground are identified
    mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')

    # Apply mask to original image
    img_array = img_array * mask2[:, :, np.newaxis]

    # Highlight cyclone boundaries
    img_array[np.where((img_array == [0, 0, 0]).all(axis=2))] = [255, 255, 255]

    plt.subplot(1, 2, 2)
    plt.imshow(img_array)
    plt.axis('off')
    plt.title('GrabCut')

    plt.tight_layout()
    plt.show()

    return img_array


In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from skimage.segmentation import slic
from skimage.graph import rag_mean_color
from skimage.graph import cut_normalized

def image_highlighting_superpixels(cropped_image):
    img_array = np.array(cropped_image)

    # Convert image to LAB color space
    lab_image = cv2.cvtColor(img_array, cv2.COLOR_BGR2LAB)

    # Perform superpixel segmentation
    segments = slic(lab_image, n_segments=500, compactness=10, sigma=1)

    # Create graph from superpixels
    g = rag_mean_color(lab_image, segments)

    # Perform graph cuts to extract foreground
    labels = cut_normalized(segments, g)

    # Highlight cyclone boundaries
    img_array[labels == 0] = [0, 0, 0]

    plt.subplot(1, 2, 2)
    plt.imshow(img_array)
    plt.axis('off')
    plt.title('Superpixels')

    plt.tight_layout()
    plt.show()

    return img_array


In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from skimage.segmentation import felzenszwalb

def image_highlighting_felzenszwalb(cropped_image):
    img_array = np.array(cropped_image)

    # Convert image to RGB color space
    rgb_image = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)

    # Perform Felzenszwalb's algorithm for segmentation
    segments = felzenszwalb(rgb_image, scale=100, sigma=0.3, min_size=100)

    # Highlight cyclone boundaries
    img_array[segments == 0] = [255, 87, 51]

    plt.subplot(1, 2, 2)
    plt.imshow(img_array)
    plt.axis('off')
    plt.title('Felzenszwalb')

    plt.tight_layout()
    plt.show()

    return img_array


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage import color
from tensorflow.keras.applications.resnet50 import preprocess_input

def image_highlighting_resnet(cropped_image):
    # Convert the image to a NumPy array
    img_array = np.array(cropped_image)

    # Ensure the image is in RGB format
    if len(img_array.shape) == 2:  # Grayscale image
        img_array = cv2.cvtColor(img_array, cv2.COLOR_GRAY2RGB)
    elif img_array.shape[2] == 4:  # RGBA image
        img_array = cv2.cvtColor(img_array, cv2.COLOR_RGBA2RGB)

    # Resize the image to match the input size of ResNet-50
    resized_image = cv2.resize(img_array, (224, 224))

    # Preprocess the image for ResNet-50
    preprocessed_image = preprocess_input(resized_image)

    # Add batch dimension to the preprocessed image
    preprocessed_image = np.expand_dims(preprocessed_image, axis=0)

    # Load the ResNet-50 model
    model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False)

    # Encode the image using ResNet-50
    features = model.predict(preprocessed_image)

    # Perform segmentation decoding and highlighting
    # ... Add your segmentation decoding and highlighting code here ...
    # Here's an example code snippet for highlighting the image using watershed segmentation:

    single_channel = img_array[:, :, 0]

    ret1, thresh = cv2.threshold(single_channel, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    kernel = np.ones((3, 3), np.uint8)
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)

    opening = clear_border(opening)
    sure_bg = cv2.dilate(opening, kernel, iterations=10)
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    ret2, sure_fg = cv2.threshold(dist_transform, 0.5 * dist_transform.max(), 255, 0)
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg, sure_fg)

    ret3, markers = cv2.connectedComponents(sure_fg)
    markers = markers + 10
    markers[unknown == 255] = 0

    markers = cv2.watershed(img_array, markers)

    img_array[markers == -1] = [255, 87, 51]
    img2 = color.label2rgb(markers, bg_label=0)

    plt.subplot(1, 2, 2)
    plt.imshow(img2)
    plt.axis('off')
    plt.title('ResNet')

    plt.tight_layout()
    plt.show()

    # Return the highlighted image
    return img_array


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage import color
from sklearn.cluster import MeanShift, estimate_bandwidth

def image_segmentation_meanshift(cropped_image):
    img_array = np.array(cropped_image)
    
    # Convert the image to the Lab color space
    img_lab = cv2.cvtColor(img_array, cv2.COLOR_RGB2Lab)

    # Reshape the image to 2D array
    img_2d = img_lab.reshape((-1, 3))

    # Estimate the bandwidth for Mean Shift
    bandwidth = estimate_bandwidth(img_2d, quantile=0.2, n_samples=500)

    # Perform Mean Shift clustering
    ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
    ms.fit(img_2d)

    # Get the labels assigned to each pixel
    labels = ms.labels_

    # Reshape the labels to match the original image shape
    labels_2d = labels.reshape(img_array.shape[:2])

    img_array[labels_2d == -1] = [255, 87, 51]
    # Convert the labels to RGB image
    segmented_img = color.label2rgb(labels_2d, img_array, kind='avg', bg_label=0)

    # Display the segmented image
    plt.imshow(segmented_img)
    plt.axis('off')
    plt.title('Mean Shift Segmentation')
    plt.show()
    
    return segmented_img


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.segmentation import slic
from skimage.color import label2rgb

def image_segmentation_slic(cropped_image):
    img_array = np.array(cropped_image)

    # Perform SLIC segmentation
    segments = slic(img_array, n_segments=100, compactness=10)

    # Create a mask for each segment
    mask = np.zeros_like(img_array)

    # Iterate over each segment and assign corresponding pixels to the mask
    for segment_label in np.unique(segments):
        mask[segments == segment_label] = img_array[segments == segment_label].mean(axis=0)

    # Convert the mask to RGB image
    segmented_img = label2rgb(segments, image=mask, bg_label=0)

    # Display the segmented image
    plt.imshow(segmented_img)
    plt.axis('off')
    plt.title('SLIC Segmentation')
    plt.show()

    return segmented_img

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

def image_segmentation_kmeans(cropped_image):
    img_array = np.array(cropped_image)

    # Flatten the image to a 2D array of pixels
    pixels = img_array.reshape(-1, 3)

    # Perform K-Means clustering
    kmeans = KMeans(n_clusters=5, random_state=0)
    labels = kmeans.fit_predict(pixels)

    # Assign cluster centers as the new pixel values
    segmented_img = kmeans.cluster_centers_[labels].reshape(img_array.shape)

    # Convert the segmented image to uint8 datatype
    segmented_img = segmented_img.astype(np.uint8)

    # Display the segmented image
    plt.imshow(segmented_img)
    plt.axis('off')
    plt.title('K-Means Segmentation')
    plt.show()

    return segmented_img


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN

def image_segmentation_dbscan(cropped_image):
    img_array = np.array(cropped_image)

    # Flatten the image to a 2D array of pixels
    pixels = img_array.reshape(-1, 3)

    # Perform DBSCAN clustering
    dbscan = DBSCAN(eps=20, min_samples=100)
    labels = dbscan.fit_predict(pixels)

    # Get the number of clusters (excluding noise)
    num_clusters = len(np.unique(labels)) - 1

    # Create a mask for the segmented regions
    segmented_mask = labels.reshape(img_array.shape[:2])

    # Generate random colors for each cluster
    cluster_colors = np.random.randint(0, 256, (num_clusters, 3))

    # Apply colors to the segmented regions
    segmented_img = np.zeros_like(img_array)
    for cluster_id in range(1, num_clusters + 1):
        segmented_img[segmented_mask == cluster_id] = cluster_colors[cluster_id - 1]

    # Display the segmented image
    plt.imshow(segmented_img)
    plt.axis('off')
    plt.title('DBSCAN Segmentation')
    plt.show()

    return segmented_img


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import SpectralClustering

def image_segmentation_spectral(cropped_image):
    img_array = np.array(cropped_image)

    # Flatten the image to a 2D array of pixels
    pixels = img_array.reshape(-1, 3)

    # Perform Spectral Clustering
    spectral_clustering = SpectralClustering(n_clusters=5, affinity='nearest_neighbors', n_neighbors=10)
    labels = spectral_clustering.fit_predict(pixels)

    # Get the number of clusters
    num_clusters = len(np.unique(labels))

    # Create a mask for the segmented regions
    segmented_mask = labels.reshape(img_array.shape[:2])

    # Generate random colors for each cluster
    cluster_colors = np.random.randint(0, 256, (num_clusters, 3))

    # Apply colors to the segmented regions
    segmented_img = np.zeros_like(img_array)
    for cluster_id in range(num_clusters):
        segmented_img[segmented_mask == cluster_id] = cluster_colors[cluster_id]

    # Display the segmented image
    plt.imshow(segmented_img)
    plt.axis('off')
    plt.title('Spectral Clustering Segmentation')
    plt.show()

    return segmented_img


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def image_segmentation_canny(cropped_image):
    img_array = np.array(cropped_image)

    # Convert the image to grayscale
    gray = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)

    # Apply Canny Edge Detection
    edges = cv2.Canny(gray, 100, 200)

    # Create a mask by thresholding the edges
    mask = edges > 0

    # Apply the mask to the original image to extract the edges
    segmented_img = np.zeros_like(img_array)
    segmented_img[mask] = [255, 0, 0]  # Highlight the edges in red

    # Display the segmented image
    plt.imshow(segmented_img)
    plt.axis('off')
    plt.title('Canny Edge Detection Segmentation')
    plt.show()

    return segmented_img


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.segmentation import active_contour
from skimage.color import rgb2gray

def image_segmentation_snakes(img):
    # Convert the image to grayscale
    img_gray = rgb2gray(img)

    # Initialize the snake contour
    snake_init = np.array([[10, 10], [10, 100], [100, 100], [100, 10]])

    # Run the active contours (snakes) algorithm
    snake = active_contour(img_gray, snake_init, alpha=0.1, beta=0.1, gamma=0.01)

    # Create a blank image for visualization
    visualization = np.zeros_like(img)

    # Draw the snake contour on the visualization image
    cv2.polylines(visualization, [np.int32(snake)], isClosed=True, color=(255, 0, 0), thickness=2)

    # Display the segmented image
    plt.imshow(visualization)
    plt.axis('off')
    plt.title('Active Contours Segmentation')
    plt.show()

    return visualization


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.color import rgb2gray
from skimage.segmentation import chan_vese

def image_segmentation_level_sets(cropped_image):
    # Convert the image to grayscale
    img_gray = rgb2gray(cropped_image)

    # Apply the Chan-Vese level set algorithm
    mask = chan_vese(img_gray, mu=0.25, lambda1=1, lambda2=1, tol=1e-3)

    # Create a mask overlay on the original image
    segmented_image = np.zeros_like(cropped_image)
    segmented_image[mask] = [255, 255, 255]  # Highlight the segmented regions in red

    # Display the segmented image
    plt.imshow(segmented_image)
    plt.axis('off')
    plt.title('Level Sets Segmentation')
    plt.show()

    return segmented_image


## Saving the Pre-processed images locally

In [ ]:
def save_processed_image(img_array, filename):
    output_dir = 'Dataset/pre-processing/Level-Sets-Segmentation/'
    output_path = os.path.join(output_dir, filename)
    processed_image = Image.fromarray(img_array)
    processed_image.save(output_path)
    print(f"Processed image saved at: {output_path}")

## Cropping the required section of the images

In [ ]:
def image_crop(image, filename):
    if int(filename[11:15]) <= 2020 or 'MAY2021' in filename or 'SEP2021' in filename:
        crop_coords = (150, 300, 600, 700)
    else:
        crop_coords = (300, 600, 1200, 1300)
    cropped_image = image.crop(crop_coords)
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.axis('off')
    plt.title('Original Image')
    return cropped_image

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image, ImageFilter
import numpy as np
from skimage.segmentation import clear_border
from skimage import color

image_folder_path = 'Dataset/images/'

row = data.iloc[2]

filename = row['IR1']
image_path = image_folder_path + filename
image = Image.open(image_path)
cropped_image = image_crop(image, filename)
highlighted_image = image_highlighting(cropped_image)
highlighted_image = image_highlighting_grabcut(cropped_image)
highlighted_image = image_highlighting_superpixels(cropped_image)
highlighted_image = image_highlighting_felzenszwalb(cropped_image)
highlighted_image = image_highlighting_resnet(cropped_image)
highlighted_image = image_segmentation_meanshift(cropped_image)
highlighted_image = image_segmentation_slic(cropped_image)
highlighted_image = image_segmentation_kmeans(cropped_image)
#highlighted_image = image_segmentation_dbscan(cropped_image)
#highlighted_image = image_segmentation_spectral(cropped_image)
highlighted_image = image_segmentation_canny(cropped_image)
highlighted_image = image_segmentation_snakes(cropped_image)
highlighted_image = image_segmentation_level_sets(cropped_image)
#save_processed_image(highlighted_image, filename)

## Generating Pre-processed images

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

image_folder_path = 'Dataset/images/'

image_counter = 0
image_not_counter = 0

for index, row in data.iterrows():
    filename = row['IR1']
    
    image_path = image_folder_path + filename
    
    try:
        image = Image.open(image_path)
        cropped_image = image_crop(image, filename)
        highlighted_image = image_segmentation_level_sets(cropped_image)
        save_processed_image(highlighted_image, filename)
        image_counter += 1
    except Exception as err:
        print(err)
        image_not_counter += 1
    
print(image_counter)
print(image_not_counter)

In [ ]:
def save_processed_image(img_array, filename):
    output_dir = 'Dataset/pre-processing/K-Means-Segmentation/'
    output_path = os.path.join(output_dir, filename)
    processed_image = Image.fromarray(img_array)
    processed_image.save(output_path)
    print(f"Processed image saved at: {output_path}")

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

image_folder_path = 'Dataset/images/'

image_counter = 0
image_not_counter = 0

for index, row in data.iterrows():
    filename = row['IR1']
    
    image_path = image_folder_path + filename
    
    try:
        image = Image.open(image_path)
        cropped_image = image_crop(image, filename)
        highlighted_image = image_segmentation_kmeans(cropped_image)
        save_processed_image(highlighted_image, filename)
        image_counter += 1
    except Exception as err:
        print(err)
        image_not_counter += 1
    
print(image_counter)
print(image_not_counter)

In [ ]:
def save_processed_image(img_array, filename):
    output_dir = 'Dataset/pre-processing/SLIC-Segmentation/'
    output_path = os.path.join(output_dir, filename)
    processed_image = Image.fromarray(img_array)
    processed_image.save(output_path)
    print(f"Processed image saved at: {output_path}")

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

image_folder_path = 'Dataset/images/'

image_counter = 0
image_not_counter = 0

for index, row in data.iterrows():
    filename = row['IR1']
    
    image_path = image_folder_path + filename
    
    try:
        image = Image.open(image_path)
        cropped_image = image_crop(image, filename)
        highlighted_image = image_segmentation_slic(cropped_image)
        save_processed_image(highlighted_image, filename)
        image_counter += 1
    except Exception as err:
        print(err)
        image_not_counter += 1
    
print(image_counter)
print(image_not_counter)

In [ ]:
def save_processed_image(img_array, filename):
    output_dir = 'Dataset/pre-processing/Mean-Shift-Segmentation/'
    output_path = os.path.join(output_dir, filename)
    processed_image = Image.fromarray(img_array)
    processed_image.save(output_path)
    print(f"Processed image saved at: {output_path}")

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

image_folder_path = 'Dataset/images/'

image_counter = 0
image_not_counter = 0

for index, row in data.iterrows():
    filename = row['IR1']
    
    image_path = image_folder_path + filename
    
    try:
        image = Image.open(image_path)
        cropped_image = image_crop(image, filename)
        highlighted_image = image_segmentation_meanshift(cropped_image)
        save_processed_image(highlighted_image, filename)
        image_counter += 1
    except Exception as err:
        print(err)
        image_not_counter += 1
    
print(image_counter)
print(image_not_counter)

In [ ]:
def save_processed_image(img_array, filename):
    output_dir = 'Dataset/pre-processing/ResNet/'
    output_path = os.path.join(output_dir, filename)
    processed_image = Image.fromarray(img_array)
    processed_image.save(output_path)
    print(f"Processed image saved at: {output_path}")

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

image_folder_path = 'Dataset/images/'

image_counter = 0
image_not_counter = 0

for index, row in data.iterrows():
    filename = row['IR1']
    
    image_path = image_folder_path + filename
    
    try:
        image = Image.open(image_path)
        cropped_image = image_crop(image, filename)
        highlighted_image = image_highlighting_resnet(cropped_image)
        save_processed_image(highlighted_image, filename)
        image_counter += 1
    except Exception as err:
        print(err)
        image_not_counter += 1
    
print(image_counter)
print(image_not_counter)

## Loading the pre-processed images into dataframes

In [ ]:
import tensorflow as tf
import pandas as pd
import os

def load_and_preprocess_image(image_path):
    try:
        image = tf.io.read_file('Dataset/processed_images/' + image_path)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, (256, 256))
        image = image / 255.0
        return image, image_path
    except Exception as err:
        print(err)
        return None, image_path

data['image'], data['IR1'] = zip(*data['IR1'].apply(load_and_preprocess_image))

data = data.dropna(subset=['image'])

images = tf.stack(list(data['image'].values))
labels = data['WMO_WIND'].values.astype(float)

filenames = data['IR1'].values

dataset_display = tf.data.Dataset.from_tensor_slices((images, labels, filenames))
dataset = tf.data.Dataset.from_tensor_slices((images, labels))

## Displaying the dataset

In [ ]:
import matplotlib.pyplot as plt

num_images = 10
image_samples = dataset_display.take(num_images)

for image, label, image_filename in image_samples:
    wind_value = label.numpy() 
    plt.imshow(image)
    plt.title(f"Wind Value: {wind_value}\nFilename: {image_filename}")
    plt.axis('off')
    plt.show()

## Splitting the data

In [ ]:
train_size = int(0.8 * len(dataset))
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

batch_size = 32
train_dataset = train_dataset.shuffle(train_size).batch(batch_size).prefetch(1)
val_dataset = val_dataset.batch(batch_size).prefetch(1)

## Data Augmentation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_augmentation = ImageDataGenerator(
    rotation_range=20,  # Randomly rotate images within the range (-20, 20) degrees
    width_shift_range=0.1,  # Randomly shift the width of images by 0-10% of the total width
    height_shift_range=0.1,  # Randomly shift the height of images by 0-10% of the total height
    horizontal_flip=True,  # Randomly flip images horizontally
    zoom_range=0.1  # Randomly zoom images by 0-10%
)

# K-Fold Cross Validation Simple CNN

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Sequential
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.regularizers import l1, l2
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


k = 5

skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

images = np.array(images)
labels = np.array(labels)

for train_index, val_index in skf.split(images, labels):
    train_dataset = tf.data.Dataset.from_tensor_slices((images[train_index], labels[train_index]))
    val_dataset = tf.data.Dataset.from_tensor_slices((images[val_index], labels[val_index]))
    
    train_dataset = train_dataset.shuffle(1000).batch(32)
    val_dataset = val_dataset.batch(32)
    
    data_augmentation = ImageDataGenerator(
    rotation_range=20,  # Randomly rotate images within the range (-20, 20) degrees
    width_shift_range=0.1,  # Randomly shift the width of images by 0-10% of the total width
    height_shift_range=0.1,  # Randomly shift the height of images by 0-10% of the total height
    horizontal_flip=True,  # Randomly flip images horizontally
    zoom_range=0.1  # Randomly zoom images by 0-10%
    )
    
    model = Sequential()
    model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
    model.add(MaxPooling2D())
    model.add(Conv2D(32, (3,3), 1, activation='relu'))
    model.add(MaxPooling2D())
    model.add(Conv2D(16, (3,3), 1, activation='relu'))
    model.add(MaxPooling2D())
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mae', metrics=[tf.keras.metrics.RootMeanSquaredError()])
    model.summary()
    
    model.fit(train_dataset, epochs=20, verbose=0)
    
    train_loss, train_accuracy = model.evaluate(train_dataset, verbose=0)
    val_loss, val_accuracy = model.evaluate(val_dataset, verbose=0)
    
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

print('Average training loss:', np.mean(train_losses))
print('Average training accuracy:', np.mean(train_accuracies))
print('Training losses:', train_losses)
print('Training accuracies:', train_accuracies)
print('Average validation loss:', np.mean(val_losses))
print('Average validation accuracy:', np.mean(val_accuracies))
print('Validation losses:', val_losses)
print('Validation accuracies:', val_accuracies)

# Level Set Segmentation

In [ ]:
import pandas as pd

data = pd.read_csv("Dataset/Dataset.csv")
data.head()

features = ["SEASON", "SUBBASIN", "ISO_TIME", "WMO_WIND"]
#features.append("WMO_PRES")
data = data[features]
data.head()

from datetime import datetime

date_series = data["ISO_TIME"]
datetime_series = pd.to_datetime(date_series, format="%d-%m-%Y %H:%M")
formatted_series = datetime_series.dt.strftime("%d%b%Y_%H%M").str.upper()
file_name = "3DIMG_" + formatted_series + "_L1C_ASIA_MER_IR1_V01R00.jpg"

file_name.head()

data["IR1"] = file_name

data.head()

data['WMO_WIND'] = data['WMO_WIND'].replace(' ', 0)
data['WMO_WIND'] = pd.to_numeric(data['WMO_WIND'], errors='coerce')

import tensorflow as tf
import pandas as pd
import os

def load_and_preprocess_image(image_path):
    try:
        image = tf.io.read_file('Dataset/pre-processing/Level-Sets-Segmentation/' + image_path)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, (256, 256))
        image = image / 255.0
        return image, image_path
    except Exception as err:
        print(err)
        return None, image_path

data['image'], data['IR1'] = zip(*data['IR1'].apply(load_and_preprocess_image))

data = data.dropna(subset=['image'])

images = tf.stack(list(data['image'].values))
labels = data['WMO_WIND'].values.astype(float)

filenames = data['IR1'].values

dataset_display = tf.data.Dataset.from_tensor_slices((images, labels, filenames))
dataset = tf.data.Dataset.from_tensor_slices((images, labels))

import matplotlib.pyplot as plt

num_images = 10
image_samples = dataset_display.take(num_images)

for image, label, image_filename in image_samples:
    wind_value = label.numpy() 
    plt.imshow(image)
    plt.title(f"Wind Value: {wind_value}\nFilename: {image_filename}")
    plt.axis('off')
    plt.show()
    
train_size = int(0.8 * len(dataset))
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

batch_size = 32
train_dataset = train_dataset.shuffle(train_size).batch(batch_size).prefetch(1)
val_dataset = val_dataset.batch(batch_size).prefetch(1)

from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_augmentation = ImageDataGenerator(
    rotation_range=20,  # Randomly rotate images within the range (-20, 20) degrees
    width_shift_range=0.1,  # Randomly shift the width of images by 0-10% of the total width
    height_shift_range=0.1,  # Randomly shift the height of images by 0-10% of the total height
    horizontal_flip=True,  # Randomly flip images horizontally
    zoom_range=0.1  # Randomly zoom images by 0-10%
)



import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Sequential
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.regularizers import l1, l2
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


k = 5

skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

images = np.array(images)
labels = np.array(labels)

for train_index, val_index in skf.split(images, labels):
    train_dataset = tf.data.Dataset.from_tensor_slices((images[train_index], labels[train_index]))
    val_dataset = tf.data.Dataset.from_tensor_slices((images[val_index], labels[val_index]))
    
    train_dataset = train_dataset.shuffle(1000).batch(32)
    val_dataset = val_dataset.batch(32)
    
    data_augmentation = ImageDataGenerator(
    rotation_range=20,  # Randomly rotate images within the range (-20, 20) degrees
    width_shift_range=0.1,  # Randomly shift the width of images by 0-10% of the total width
    height_shift_range=0.1,  # Randomly shift the height of images by 0-10% of the total height
    horizontal_flip=True,  # Randomly flip images horizontally
    zoom_range=0.1  # Randomly zoom images by 0-10%
    )
    
    model = Sequential()
    model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
    model.add(MaxPooling2D())
    model.add(Conv2D(32, (3,3), 1, activation='relu'))
    model.add(MaxPooling2D())
    model.add(Conv2D(16, (3,3), 1, activation='relu'))
    model.add(MaxPooling2D())
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mae', metrics=[tf.keras.metrics.RootMeanSquaredError()])
    model.summary()
    
    model.fit(train_dataset, epochs=20, verbose=0)
    
    train_loss, train_accuracy = model.evaluate(train_dataset, verbose=0)
    val_loss, val_accuracy = model.evaluate(val_dataset, verbose=0)
    
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

print('Average training loss:', np.mean(train_losses))
print('Average training accuracy:', np.mean(train_accuracies))
print('Training losses:', train_losses)
print('Training accuracies:', train_accuracies)
print('Average validation loss:', np.mean(val_losses))
print('Average validation accuracy:', np.mean(val_accuracies))
print('Validation losses:', val_losses)
print('Validation accuracies:', val_accuracies)

# K-Means Segmentation

In [ ]:
import pandas as pd

data = pd.read_csv("Dataset/Dataset.csv")
data.head()

features = ["SEASON", "SUBBASIN", "ISO_TIME", "WMO_WIND"]
#features.append("WMO_PRES")
data = data[features]
data.head()

from datetime import datetime

date_series = data["ISO_TIME"]
datetime_series = pd.to_datetime(date_series, format="%d-%m-%Y %H:%M")
formatted_series = datetime_series.dt.strftime("%d%b%Y_%H%M").str.upper()
file_name = "3DIMG_" + formatted_series + "_L1C_ASIA_MER_IR1_V01R00.jpg"

file_name.head()

data["IR1"] = file_name

data.head()

data['WMO_WIND'] = data['WMO_WIND'].replace(' ', 0)
data['WMO_WIND'] = pd.to_numeric(data['WMO_WIND'], errors='coerce')

import tensorflow as tf
import pandas as pd
import os

def load_and_preprocess_image(image_path):
    try:
        image = tf.io.read_file('Dataset/pre-processing/K-Means-Segmentation/' + image_path)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, (256, 256))
        image = image / 255.0
        return image, image_path
    except Exception as err:
        print(err)
        return None, image_path

data['image'], data['IR1'] = zip(*data['IR1'].apply(load_and_preprocess_image))

data = data.dropna(subset=['image'])

images = tf.stack(list(data['image'].values))
labels = data['WMO_WIND'].values.astype(float)

filenames = data['IR1'].values

dataset_display = tf.data.Dataset.from_tensor_slices((images, labels, filenames))
dataset = tf.data.Dataset.from_tensor_slices((images, labels))

import matplotlib.pyplot as plt

num_images = 10
image_samples = dataset_display.take(num_images)

for image, label, image_filename in image_samples:
    wind_value = label.numpy() 
    plt.imshow(image)
    plt.title(f"Wind Value: {wind_value}\nFilename: {image_filename}")
    plt.axis('off')
    plt.show()
    
train_size = int(0.8 * len(dataset))
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

batch_size = 32
train_dataset = train_dataset.shuffle(train_size).batch(batch_size).prefetch(1)
val_dataset = val_dataset.batch(batch_size).prefetch(1)

from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_augmentation = ImageDataGenerator(
    rotation_range=20,  # Randomly rotate images within the range (-20, 20) degrees
    width_shift_range=0.1,  # Randomly shift the width of images by 0-10% of the total width
    height_shift_range=0.1,  # Randomly shift the height of images by 0-10% of the total height
    horizontal_flip=True,  # Randomly flip images horizontally
    zoom_range=0.1  # Randomly zoom images by 0-10%
)



import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Sequential
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.regularizers import l1, l2
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


k = 5

skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

images = np.array(images)
labels = np.array(labels)

for train_index, val_index in skf.split(images, labels):
    train_dataset = tf.data.Dataset.from_tensor_slices((images[train_index], labels[train_index]))
    val_dataset = tf.data.Dataset.from_tensor_slices((images[val_index], labels[val_index]))
    
    train_dataset = train_dataset.shuffle(1000).batch(32)
    val_dataset = val_dataset.batch(32)
    
    data_augmentation = ImageDataGenerator(
    rotation_range=20,  # Randomly rotate images within the range (-20, 20) degrees
    width_shift_range=0.1,  # Randomly shift the width of images by 0-10% of the total width
    height_shift_range=0.1,  # Randomly shift the height of images by 0-10% of the total height
    horizontal_flip=True,  # Randomly flip images horizontally
    zoom_range=0.1  # Randomly zoom images by 0-10%
    )
    
    model = Sequential()
    model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
    model.add(MaxPooling2D())
    model.add(Conv2D(32, (3,3), 1, activation='relu'))
    model.add(MaxPooling2D())
    model.add(Conv2D(16, (3,3), 1, activation='relu'))
    model.add(MaxPooling2D())
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mae', metrics=[tf.keras.metrics.RootMeanSquaredError()])
    model.summary()
    
    model.fit(train_dataset, epochs=20, verbose=0)
    
    train_loss, train_accuracy = model.evaluate(train_dataset, verbose=0)
    val_loss, val_accuracy = model.evaluate(val_dataset, verbose=0)
    
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

print('Average training loss:', np.mean(train_losses))
print('Average training accuracy:', np.mean(train_accuracies))
print('Training losses:', train_losses)
print('Training accuracies:', train_accuracies)
print('Average validation loss:', np.mean(val_losses))
print('Average validation accuracy:', np.mean(val_accuracies))
print('Validation losses:', val_losses)
print('Validation accuracies:', val_accuracies)

# Mean Shift Segmentation

In [ ]:
import pandas as pd

data = pd.read_csv("Dataset/Dataset.csv")
data.head()

features = ["SEASON", "SUBBASIN", "ISO_TIME", "WMO_WIND"]
#features.append("WMO_PRES")
data = data[features]
data.head()

from datetime import datetime

date_series = data["ISO_TIME"]
datetime_series = pd.to_datetime(date_series, format="%d-%m-%Y %H:%M")
formatted_series = datetime_series.dt.strftime("%d%b%Y_%H%M").str.upper()
file_name = "3DIMG_" + formatted_series + "_L1C_ASIA_MER_IR1_V01R00.jpg"

file_name.head()

data["IR1"] = file_name

data.head()

data['WMO_WIND'] = data['WMO_WIND'].replace(' ', 0)
data['WMO_WIND'] = pd.to_numeric(data['WMO_WIND'], errors='coerce')

import tensorflow as tf
import pandas as pd
import os

def load_and_preprocess_image(image_path):
    try:
        image = tf.io.read_file('Dataset/pre-processing/Mean-Shift-Segmentation/' + image_path)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, (256, 256))
        image = image / 255.0
        return image, image_path
    except Exception as err:
        print(err)
        return None, image_path

data['image'], data['IR1'] = zip(*data['IR1'].apply(load_and_preprocess_image))

data = data.dropna(subset=['image'])

images = tf.stack(list(data['image'].values))
labels = data['WMO_WIND'].values.astype(float)

filenames = data['IR1'].values

dataset_display = tf.data.Dataset.from_tensor_slices((images, labels, filenames))
dataset = tf.data.Dataset.from_tensor_slices((images, labels))

import matplotlib.pyplot as plt

num_images = 10
image_samples = dataset_display.take(num_images)

for image, label, image_filename in image_samples:
    wind_value = label.numpy() 
    plt.imshow(image)
    plt.title(f"Wind Value: {wind_value}\nFilename: {image_filename}")
    plt.axis('off')
    plt.show()
    
train_size = int(0.8 * len(dataset))
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

batch_size = 32
train_dataset = train_dataset.shuffle(train_size).batch(batch_size).prefetch(1)
val_dataset = val_dataset.batch(batch_size).prefetch(1)

from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_augmentation = ImageDataGenerator(
    rotation_range=20,  # Randomly rotate images within the range (-20, 20) degrees
    width_shift_range=0.1,  # Randomly shift the width of images by 0-10% of the total width
    height_shift_range=0.1,  # Randomly shift the height of images by 0-10% of the total height
    horizontal_flip=True,  # Randomly flip images horizontally
    zoom_range=0.1  # Randomly zoom images by 0-10%
)



import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Sequential
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.regularizers import l1, l2
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


k = 5

skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

images = np.array(images)
labels = np.array(labels)

for train_index, val_index in skf.split(images, labels):
    train_dataset = tf.data.Dataset.from_tensor_slices((images[train_index], labels[train_index]))
    val_dataset = tf.data.Dataset.from_tensor_slices((images[val_index], labels[val_index]))
    
    train_dataset = train_dataset.shuffle(1000).batch(32)
    val_dataset = val_dataset.batch(32)
    
    data_augmentation = ImageDataGenerator(
    rotation_range=20,  # Randomly rotate images within the range (-20, 20) degrees
    width_shift_range=0.1,  # Randomly shift the width of images by 0-10% of the total width
    height_shift_range=0.1,  # Randomly shift the height of images by 0-10% of the total height
    horizontal_flip=True,  # Randomly flip images horizontally
    zoom_range=0.1  # Randomly zoom images by 0-10%
    )
    
    model = Sequential()
    model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
    model.add(MaxPooling2D())
    model.add(Conv2D(32, (3,3), 1, activation='relu'))
    model.add(MaxPooling2D())
    model.add(Conv2D(16, (3,3), 1, activation='relu'))
    model.add(MaxPooling2D())
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mae', metrics=[tf.keras.metrics.RootMeanSquaredError()])
    model.summary()
    
    model.fit(train_dataset, epochs=20, verbose=0)
    
    train_loss, train_accuracy = model.evaluate(train_dataset, verbose=0)
    val_loss, val_accuracy = model.evaluate(val_dataset, verbose=0)
    
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

print('Average training loss:', np.mean(train_losses))
print('Average training accuracy:', np.mean(train_accuracies))
print('Training losses:', train_losses)
print('Training accuracies:', train_accuracies)
print('Average validation loss:', np.mean(val_losses))
print('Average validation accuracy:', np.mean(val_accuracies))
print('Validation losses:', val_losses)
print('Validation accuracies:', val_accuracies)

# ResNet

In [ ]:
import pandas as pd

data = pd.read_csv("Dataset/Dataset.csv")
data.head()

features = ["SEASON", "SUBBASIN", "ISO_TIME", "WMO_WIND"]
#features.append("WMO_PRES")
data = data[features]
data.head()

from datetime import datetime

date_series = data["ISO_TIME"]
datetime_series = pd.to_datetime(date_series, format="%d-%m-%Y %H:%M")
formatted_series = datetime_series.dt.strftime("%d%b%Y_%H%M").str.upper()
file_name = "3DIMG_" + formatted_series + "_L1C_ASIA_MER_IR1_V01R00.jpg"

file_name.head()

data["IR1"] = file_name

data.head()

data['WMO_WIND'] = data['WMO_WIND'].replace(' ', 0)
data['WMO_WIND'] = pd.to_numeric(data['WMO_WIND'], errors='coerce')

import tensorflow as tf
import pandas as pd
import os

def load_and_preprocess_image(image_path):
    try:
        image = tf.io.read_file('Dataset/pre-processing/ResNet/' + image_path)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, (256, 256))
        image = image / 255.0
        return image, image_path
    except Exception as err:
        print(err)
        return None, image_path

data['image'], data['IR1'] = zip(*data['IR1'].apply(load_and_preprocess_image))

data = data.dropna(subset=['image'])

images = tf.stack(list(data['image'].values))
labels = data['WMO_WIND'].values.astype(float)

filenames = data['IR1'].values

dataset_display = tf.data.Dataset.from_tensor_slices((images, labels, filenames))
dataset = tf.data.Dataset.from_tensor_slices((images, labels))

import matplotlib.pyplot as plt

num_images = 10
image_samples = dataset_display.take(num_images)

for image, label, image_filename in image_samples:
    wind_value = label.numpy() 
    plt.imshow(image)
    plt.title(f"Wind Value: {wind_value}\nFilename: {image_filename}")
    plt.axis('off')
    plt.show()
    
train_size = int(0.8 * len(dataset))
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

batch_size = 32
train_dataset = train_dataset.shuffle(train_size).batch(batch_size).prefetch(1)
val_dataset = val_dataset.batch(batch_size).prefetch(1)

from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_augmentation = ImageDataGenerator(
    rotation_range=20,  # Randomly rotate images within the range (-20, 20) degrees
    width_shift_range=0.1,  # Randomly shift the width of images by 0-10% of the total width
    height_shift_range=0.1,  # Randomly shift the height of images by 0-10% of the total height
    horizontal_flip=True,  # Randomly flip images horizontally
    zoom_range=0.1  # Randomly zoom images by 0-10%
)



import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Sequential
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.regularizers import l1, l2
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


k = 5

skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

images = np.array(images)
labels = np.array(labels)

for train_index, val_index in skf.split(images, labels):
    train_dataset = tf.data.Dataset.from_tensor_slices((images[train_index], labels[train_index]))
    val_dataset = tf.data.Dataset.from_tensor_slices((images[val_index], labels[val_index]))
    
    train_dataset = train_dataset.shuffle(1000).batch(32)
    val_dataset = val_dataset.batch(32)
    
    data_augmentation = ImageDataGenerator(
    rotation_range=20,  # Randomly rotate images within the range (-20, 20) degrees
    width_shift_range=0.1,  # Randomly shift the width of images by 0-10% of the total width
    height_shift_range=0.1,  # Randomly shift the height of images by 0-10% of the total height
    horizontal_flip=True,  # Randomly flip images horizontally
    zoom_range=0.1  # Randomly zoom images by 0-10%
    )
    
    model = Sequential()
    model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
    model.add(MaxPooling2D())
    model.add(Conv2D(32, (3,3), 1, activation='relu'))
    model.add(MaxPooling2D())
    model.add(Conv2D(16, (3,3), 1, activation='relu'))
    model.add(MaxPooling2D())
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mae', metrics=[tf.keras.metrics.RootMeanSquaredError()])
    model.summary()
    
    model.fit(train_dataset, epochs=20, verbose=0)
    
    train_loss, train_accuracy = model.evaluate(train_dataset, verbose=0)
    val_loss, val_accuracy = model.evaluate(val_dataset, verbose=0)
    
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

print('Average training loss:', np.mean(train_losses))
print('Average training accuracy:', np.mean(train_accuracies))
print('Training losses:', train_losses)
print('Training accuracies:', train_accuracies)
print('Average validation loss:', np.mean(val_losses))
print('Average validation accuracy:', np.mean(val_accuracies))
print('Validation losses:', val_losses)
print('Validation accuracies:', val_accuracies)

# Level Set Overfitting

In [ ]:
import pandas as pd

data = pd.read_csv("Dataset/Dataset.csv")
data.head()

features = ["SEASON", "SUBBASIN", "ISO_TIME", "WMO_WIND"]
#features.append("WMO_PRES")
data = data[features]
data.head()

from datetime import datetime

date_series = data["ISO_TIME"]
datetime_series = pd.to_datetime(date_series, format="%d-%m-%Y %H:%M")
formatted_series = datetime_series.dt.strftime("%d%b%Y_%H%M").str.upper()
file_name = "3DIMG_" + formatted_series + "_L1C_ASIA_MER_IR1_V01R00.jpg"

file_name.head()

data["IR1"] = file_name

data.head()

data['WMO_WIND'] = data['WMO_WIND'].replace(' ', 0)
data['WMO_WIND'] = pd.to_numeric(data['WMO_WIND'], errors='coerce')

import tensorflow as tf
import pandas as pd
import os

def load_and_preprocess_image(image_path):
    try:
        image = tf.io.read_file('Dataset/pre-processing/Level-Sets-Segmentation/' + image_path)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, (256, 256))
        image = image / 255.0
        return image, image_path
    except Exception as err:
        print(err)
        return None, image_path

data['image'], data['IR1'] = zip(*data['IR1'].apply(load_and_preprocess_image))

data = data.dropna(subset=['image'])

images = tf.stack(list(data['image'].values))
labels = data['WMO_WIND'].values.astype(float)

filenames = data['IR1'].values

dataset_display = tf.data.Dataset.from_tensor_slices((images, labels, filenames))
dataset = tf.data.Dataset.from_tensor_slices((images, labels))

import matplotlib.pyplot as plt

num_images = 10
image_samples = dataset_display.take(num_images)

for image, label, image_filename in image_samples:
    wind_value = label.numpy() 
    plt.imshow(image)
    plt.title(f"Wind Value: {wind_value}\nFilename: {image_filename}")
    plt.axis('off')
    plt.show()
    
train_size = int(0.8 * len(dataset))
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

batch_size = 32
train_dataset = train_dataset.shuffle(train_size).batch(batch_size).prefetch(1)
val_dataset = val_dataset.batch(batch_size).prefetch(1)

from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_augmentation = ImageDataGenerator(
    rotation_range=20,  # Randomly rotate images within the range (-20, 20) degrees
    width_shift_range=0.1,  # Randomly shift the width of images by 0-10% of the total width
    height_shift_range=0.1,  # Randomly shift the height of images by 0-10% of the total height
    horizontal_flip=True,  # Randomly flip images horizontally
    zoom_range=0.1  # Randomly zoom images by 0-10%
)



import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Sequential
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.regularizers import l1, l2
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


k = 5

skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

images = np.array(images)
labels = np.array(labels)

for train_index, val_index in skf.split(images, labels):
    train_dataset = tf.data.Dataset.from_tensor_slices((images[train_index], labels[train_index]))
    val_dataset = tf.data.Dataset.from_tensor_slices((images[val_index], labels[val_index]))
    
    train_dataset = train_dataset.shuffle(1000).batch(32)
    val_dataset = val_dataset.batch(32)
    
    data_augmentation = ImageDataGenerator(
    rotation_range=20,  # Randomly rotate images within the range (-20, 20) degrees
    width_shift_range=0.1,  # Randomly shift the width of images by 0-10% of the total width
    height_shift_range=0.1,  # Randomly shift the height of images by 0-10% of the total height
    horizontal_flip=True,  # Randomly flip images horizontally
    zoom_range=0.1  # Randomly zoom images by 0-10%
    )
    
    model = Sequential()
    model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
    model.add(Dropout(0.2))
    model.add(MaxPooling2D())
    model.add(Conv2D(32, (3,3), 1, activation='relu'))
    model.add(MaxPooling2D())
    model.add(Conv2D(16, (3,3), 1, activation='relu'))
    model.add(MaxPooling2D())
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mae', metrics=[tf.keras.metrics.RootMeanSquaredError()])
    model.summary()
    
    model.fit(train_dataset, epochs=20, verbose=0)
    
    train_loss, train_accuracy = model.evaluate(train_dataset, verbose=0)
    val_loss, val_accuracy = model.evaluate(val_dataset, verbose=0)
    
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

print('Average training loss:', np.mean(train_losses))
print('Average training accuracy:', np.mean(train_accuracies))
print('Training losses:', train_losses)
print('Training accuracies:', train_accuracies)
print('Average validation loss:', np.mean(val_losses))
print('Average validation accuracy:', np.mean(val_accuracies))
print('Validation losses:', val_losses)
print('Validation accuracies:', val_accuracies)

# Mean Shift Overfitting

In [ ]:
import pandas as pd

data = pd.read_csv("Dataset/Dataset.csv")
data.head()

features = ["SEASON", "SUBBASIN", "ISO_TIME", "WMO_WIND"]
#features.append("WMO_PRES")
data = data[features]
data.head()

from datetime import datetime

date_series = data["ISO_TIME"]
datetime_series = pd.to_datetime(date_series, format="%d-%m-%Y %H:%M")
formatted_series = datetime_series.dt.strftime("%d%b%Y_%H%M").str.upper()
file_name = "3DIMG_" + formatted_series + "_L1C_ASIA_MER_IR1_V01R00.jpg"

file_name.head()

data["IR1"] = file_name

data.head()

data['WMO_WIND'] = data['WMO_WIND'].replace(' ', 0)
data['WMO_WIND'] = pd.to_numeric(data['WMO_WIND'], errors='coerce')

import tensorflow as tf
import pandas as pd
import os

def load_and_preprocess_image(image_path):
    try:
        image = tf.io.read_file('Dataset/pre-processing/Mean-Shift-Segmentation/' + image_path)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, (256, 256))
        image = image / 255.0
        return image, image_path
    except Exception as err:
        print(err)
        return None, image_path

data['image'], data['IR1'] = zip(*data['IR1'].apply(load_and_preprocess_image))

data = data.dropna(subset=['image'])

images = tf.stack(list(data['image'].values))
labels = data['WMO_WIND'].values.astype(float)

filenames = data['IR1'].values

dataset_display = tf.data.Dataset.from_tensor_slices((images, labels, filenames))
dataset = tf.data.Dataset.from_tensor_slices((images, labels))

import matplotlib.pyplot as plt

num_images = 10
image_samples = dataset_display.take(num_images)

for image, label, image_filename in image_samples:
    wind_value = label.numpy() 
    plt.imshow(image)
    plt.title(f"Wind Value: {wind_value}\nFilename: {image_filename}")
    plt.axis('off')
    plt.show()
    
train_size = int(0.8 * len(dataset))
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

batch_size = 32
train_dataset = train_dataset.shuffle(train_size).batch(batch_size).prefetch(1)
val_dataset = val_dataset.batch(batch_size).prefetch(1)

from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_augmentation = ImageDataGenerator(
    rotation_range=20,  # Randomly rotate images within the range (-20, 20) degrees
    width_shift_range=0.1,  # Randomly shift the width of images by 0-10% of the total width
    height_shift_range=0.1,  # Randomly shift the height of images by 0-10% of the total height
    horizontal_flip=True,  # Randomly flip images horizontally
    zoom_range=0.1  # Randomly zoom images by 0-10%
)



import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Sequential
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.regularizers import l1, l2
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


k = 5

skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

images = np.array(images)
labels = np.array(labels)

for train_index, val_index in skf.split(images, labels):
    train_dataset = tf.data.Dataset.from_tensor_slices((images[train_index], labels[train_index]))
    val_dataset = tf.data.Dataset.from_tensor_slices((images[val_index], labels[val_index]))
    
    train_dataset = train_dataset.shuffle(1000).batch(32)
    val_dataset = val_dataset.batch(32)
    
    data_augmentation = ImageDataGenerator(
    rotation_range=20,  # Randomly rotate images within the range (-20, 20) degrees
    width_shift_range=0.1,  # Randomly shift the width of images by 0-10% of the total width
    height_shift_range=0.1,  # Randomly shift the height of images by 0-10% of the total height
    horizontal_flip=True,  # Randomly flip images horizontally
    zoom_range=0.1  # Randomly zoom images by 0-10%
    )
    
    model = Sequential()
    model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
    model.add(Dropout(0.2))
    model.add(MaxPooling2D())
    model.add(Conv2D(32, (3,3), 1, activation='relu'))
    model.add(MaxPooling2D())
    model.add(Conv2D(16, (3,3), 1, activation='relu'))
    model.add(MaxPooling2D())
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mae', metrics=[tf.keras.metrics.RootMeanSquaredError()])
    model.summary()
    
    model.fit(train_dataset, epochs=20, verbose=0)
    
    train_loss, train_accuracy = model.evaluate(train_dataset, verbose=0)
    val_loss, val_accuracy = model.evaluate(val_dataset, verbose=0)
    
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

print('Average training loss:', np.mean(train_losses))
print('Average training accuracy:', np.mean(train_accuracies))
print('Training losses:', train_losses)
print('Training accuracies:', train_accuracies)
print('Average validation loss:', np.mean(val_losses))
print('Average validation accuracy:', np.mean(val_accuracies))
print('Validation losses:', val_losses)
print('Validation accuracies:', val_accuracies)

In [ ]:
import pandas as pd

data = pd.read_csv("Dataset/Dataset.csv")
data.head()

features = ["SEASON", "SUBBASIN", "ISO_TIME", "WMO_WIND"]
#features.append("WMO_PRES")
data = data[features]
data.head()

from datetime import datetime

date_series = data["ISO_TIME"]
datetime_series = pd.to_datetime(date_series, format="%d-%m-%Y %H:%M")
formatted_series = datetime_series.dt.strftime("%d%b%Y_%H%M").str.upper()
file_name = "3DIMG_" + formatted_series + "_L1C_ASIA_MER_IR1_V01R00.jpg"

file_name.head()

data["IR1"] = file_name

data.head()

data['WMO_WIND'] = data['WMO_WIND'].replace(' ', 0)
data['WMO_WIND'] = pd.to_numeric(data['WMO_WIND'], errors='coerce')

import tensorflow as tf
import pandas as pd
import os

def load_and_preprocess_image(image_path):
    try:
        image = tf.io.read_file('Dataset/pre-processing/Level-Sets-Segmentation/' + image_path)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, (256, 256))
        image = image / 255.0
        return image, image_path
    except Exception as err:
        print(err)
        return None, image_path

data['image'], data['IR1'] = zip(*data['IR1'].apply(load_and_preprocess_image))

data = data.dropna(subset=['image'])

images = tf.stack(list(data['image'].values))
labels = data['WMO_WIND'].values.astype(float)

filenames = data['IR1'].values

dataset_display = tf.data.Dataset.from_tensor_slices((images, labels, filenames))
dataset = tf.data.Dataset.from_tensor_slices((images, labels))

import matplotlib.pyplot as plt

num_images = 10
image_samples = dataset_display.take(num_images)

for image, label, image_filename in image_samples:
    wind_value = label.numpy() 
    plt.imshow(image)
    plt.title(f"Wind Value: {wind_value}\nFilename: {image_filename}")
    plt.axis('off')
    plt.show()
    
train_size = int(0.8 * len(dataset))
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

batch_size = 32
train_dataset = train_dataset.shuffle(train_size).batch(batch_size).prefetch(1)
val_dataset = val_dataset.batch(batch_size).prefetch(1)

from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_augmentation = ImageDataGenerator(
    rotation_range=20,  # Randomly rotate images within the range (-20, 20) degrees
    width_shift_range=0.1,  # Randomly shift the width of images by 0-10% of the total width
    height_shift_range=0.1,  # Randomly shift the height of images by 0-10% of the total height
    horizontal_flip=True,  # Randomly flip images horizontally
    zoom_range=0.1  # Randomly zoom images by 0-10%
)



import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Sequential
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.regularizers import l1, l2
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


k = 5

skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

images = np.array(images)
labels = np.array(labels)

for train_index, val_index in skf.split(images, labels):
    train_dataset = tf.data.Dataset.from_tensor_slices((images[train_index], labels[train_index]))
    val_dataset = tf.data.Dataset.from_tensor_slices((images[val_index], labels[val_index]))
    
    train_dataset = train_dataset.shuffle(1000).batch(32)
    val_dataset = val_dataset.batch(32)
    
    data_augmentation = ImageDataGenerator(
    rotation_range=20,  # Randomly rotate images within the range (-20, 20) degrees
    width_shift_range=0.1,  # Randomly shift the width of images by 0-10% of the total width
    height_shift_range=0.1,  # Randomly shift the height of images by 0-10% of the total height
    horizontal_flip=True,  # Randomly flip images horizontally
    zoom_range=0.1  # Randomly zoom images by 0-10%
    )
    
    model = Sequential()
    model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
    model.add(Dropout(0.2))
    model.add(MaxPooling2D())
    model.add(Conv2D(32, (3,3), 1, activation='relu'))
    model.add(Dropout(0.2))
    model.add(MaxPooling2D())
    model.add(Conv2D(16, (3,3), 1, activation='relu'))
    model.add(Dropout(0.2))
    model.add(MaxPooling2D())
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mae', metrics=[tf.keras.metrics.RootMeanSquaredError()])
    model.summary()
    
    model.fit(train_dataset, epochs=20, verbose=0)
    
    train_loss, train_accuracy = model.evaluate(train_dataset, verbose=0)
    val_loss, val_accuracy = model.evaluate(val_dataset, verbose=0)
    
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

print('Average training loss:', np.mean(train_losses))
print('Average training accuracy:', np.mean(train_accuracies))
print('Training losses:', train_losses)
print('Training accuracies:', train_accuracies)
print('Average validation loss:', np.mean(val_losses))
print('Average validation accuracy:', np.mean(val_accuracies))
print('Validation losses:', val_losses)
print('Validation accuracies:', val_accuracies)

In [ ]:
import pandas as pd

data = pd.read_csv("Dataset/Dataset.csv")
data.head()

features = ["SEASON", "SUBBASIN", "ISO_TIME", "WMO_WIND"]
#features.append("WMO_PRES")
data = data[features]
data.head()

from datetime import datetime

date_series = data["ISO_TIME"]
datetime_series = pd.to_datetime(date_series, format="%d-%m-%Y %H:%M")
formatted_series = datetime_series.dt.strftime("%d%b%Y_%H%M").str.upper()
file_name = "3DIMG_" + formatted_series + "_L1C_ASIA_MER_IR1_V01R00.jpg"

file_name.head()

data["IR1"] = file_name

data.head()

data['WMO_WIND'] = data['WMO_WIND'].replace(' ', 0)
data['WMO_WIND'] = pd.to_numeric(data['WMO_WIND'], errors='coerce')

import tensorflow as tf
import pandas as pd
import os

def load_and_preprocess_image(image_path):
    try:
        image = tf.io.read_file('Dataset/pre-processing/Mean-Shift-Segmentation/' + image_path)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, (256, 256))
        image = image / 255.0
        return image, image_path
    except Exception as err:
        print(err)
        return None, image_path

data['image'], data['IR1'] = zip(*data['IR1'].apply(load_and_preprocess_image))

data = data.dropna(subset=['image'])

images = tf.stack(list(data['image'].values))
labels = data['WMO_WIND'].values.astype(float)

filenames = data['IR1'].values

dataset_display = tf.data.Dataset.from_tensor_slices((images, labels, filenames))
dataset = tf.data.Dataset.from_tensor_slices((images, labels))

import matplotlib.pyplot as plt

num_images = 10
image_samples = dataset_display.take(num_images)

for image, label, image_filename in image_samples:
    wind_value = label.numpy() 
    plt.imshow(image)
    plt.title(f"Wind Value: {wind_value}\nFilename: {image_filename}")
    plt.axis('off')
    plt.show()
    
train_size = int(0.8 * len(dataset))
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

batch_size = 32
train_dataset = train_dataset.shuffle(train_size).batch(batch_size).prefetch(1)
val_dataset = val_dataset.batch(batch_size).prefetch(1)

from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_augmentation = ImageDataGenerator(
    rotation_range=20,  # Randomly rotate images within the range (-20, 20) degrees
    width_shift_range=0.1,  # Randomly shift the width of images by 0-10% of the total width
    height_shift_range=0.1,  # Randomly shift the height of images by 0-10% of the total height
    horizontal_flip=True,  # Randomly flip images horizontally
    zoom_range=0.1  # Randomly zoom images by 0-10%
)



import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Sequential
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.regularizers import l1, l2
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


k = 5

skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

images = np.array(images)
labels = np.array(labels)

for train_index, val_index in skf.split(images, labels):
    train_dataset = tf.data.Dataset.from_tensor_slices((images[train_index], labels[train_index]))
    val_dataset = tf.data.Dataset.from_tensor_slices((images[val_index], labels[val_index]))
    
    train_dataset = train_dataset.shuffle(1000).batch(32)
    val_dataset = val_dataset.batch(32)
    
    data_augmentation = ImageDataGenerator(
    rotation_range=20,  # Randomly rotate images within the range (-20, 20) degrees
    width_shift_range=0.1,  # Randomly shift the width of images by 0-10% of the total width
    height_shift_range=0.1,  # Randomly shift the height of images by 0-10% of the total height
    horizontal_flip=True,  # Randomly flip images horizontally
    zoom_range=0.1  # Randomly zoom images by 0-10%
    )
    
    model = Sequential()
    model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3), kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.2))
    model.add(MaxPooling2D())
    model.add(Conv2D(32, (3,3), 1, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.2))
    model.add(MaxPooling2D())
    model.add(Conv2D(16, (3,3), 1, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.2))
    model.add(MaxPooling2D())
    model.add(Flatten())
    model.add(Dense(512, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mae', metrics=[tf.keras.metrics.RootMeanSquaredError()])
    model.summary()
    
    model.fit(train_dataset, epochs=20, verbose=0)
    
    train_loss, train_accuracy = model.evaluate(train_dataset, verbose=0)
    val_loss, val_accuracy = model.evaluate(val_dataset, verbose=0)
    
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

print('Average training loss:', np.mean(train_losses))
print('Average training accuracy:', np.mean(train_accuracies))
print('Training losses:', train_losses)
print('Training accuracies:', train_accuracies)
print('Average validation loss:', np.mean(val_losses))
print('Average validation accuracy:', np.mean(val_accuracies))
print('Validation losses:', val_losses)
print('Validation accuracies:', val_accuracies)

In [ ]:
import pandas as pd

data = pd.read_csv("Dataset/Dataset.csv")
data.head()

features = ["SEASON", "SUBBASIN", "ISO_TIME", "WMO_WIND"]
#features.append("WMO_PRES")
data = data[features]
data.head()

from datetime import datetime

date_series = data["ISO_TIME"]
datetime_series = pd.to_datetime(date_series, format="%d-%m-%Y %H:%M")
formatted_series = datetime_series.dt.strftime("%d%b%Y_%H%M").str.upper()
file_name = "3DIMG_" + formatted_series + "_L1C_ASIA_MER_IR1_V01R00.jpg"

file_name.head()

data["IR1"] = file_name

data.head()

data['WMO_WIND'] = data['WMO_WIND'].replace(' ', 0)
data['WMO_WIND'] = pd.to_numeric(data['WMO_WIND'], errors='coerce')

import tensorflow as tf
import pandas as pd
import os

def load_and_preprocess_image(image_path):
    try:
        image = tf.io.read_file('Dataset/images/' + image_path)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, (256, 256))
        image = image / 255.0
        return image, image_path
    except Exception as err:
        print(err)
        return None, image_path

data['image'], data['IR1'] = zip(*data['IR1'].apply(load_and_preprocess_image))

data = data.dropna(subset=['image'])

images = tf.stack(list(data['image'].values))
labels = data['WMO_WIND'].values.astype(float)

filenames = data['IR1'].values

dataset_display = tf.data.Dataset.from_tensor_slices((images, labels, filenames))
dataset = tf.data.Dataset.from_tensor_slices((images, labels))

import matplotlib.pyplot as plt

num_images = 10
image_samples = dataset_display.take(num_images)

for image, label, image_filename in image_samples:
    wind_value = label.numpy() 
    plt.imshow(image)
    plt.title(f"Wind Value: {wind_value}\nFilename: {image_filename}")
    plt.axis('off')
    plt.show()
    
train_size = int(0.8 * len(dataset))
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

batch_size = 32
train_dataset = train_dataset.shuffle(train_size).batch(batch_size).prefetch(1)
val_dataset = val_dataset.batch(batch_size).prefetch(1)

from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_augmentation = ImageDataGenerator(
    rotation_range=20,  # Randomly rotate images within the range (-20, 20) degrees
    width_shift_range=0.1,  # Randomly shift the width of images by 0-10% of the total width
    height_shift_range=0.1,  # Randomly shift the height of images by 0-10% of the total height
    horizontal_flip=True,  # Randomly flip images horizontally
    zoom_range=0.1  # Randomly zoom images by 0-10%
)



import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Sequential
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.regularizers import l1, l2
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


k = 5

skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

images = np.array(images)
labels = np.array(labels)

for train_index, val_index in skf.split(images, labels):
    train_dataset = tf.data.Dataset.from_tensor_slices((images[train_index], labels[train_index]))
    val_dataset = tf.data.Dataset.from_tensor_slices((images[val_index], labels[val_index]))
    
    train_dataset = train_dataset.shuffle(1000).batch(32)
    val_dataset = val_dataset.batch(32)
    
    data_augmentation = ImageDataGenerator(
    rotation_range=20,  # Randomly rotate images within the range (-20, 20) degrees
    width_shift_range=0.1,  # Randomly shift the width of images by 0-10% of the total width
    height_shift_range=0.1,  # Randomly shift the height of images by 0-10% of the total height
    horizontal_flip=True,  # Randomly flip images horizontally
    zoom_range=0.1  # Randomly zoom images by 0-10%
    )
    
    model = Sequential()
    model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
    model.add(MaxPooling2D())
    model.add(Conv2D(32, (3,3), 1, activation='relu'))
    model.add(MaxPooling2D())
    model.add(Conv2D(16, (3,3), 1, activation='relu'))
    model.add(MaxPooling2D())
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mae', metrics=[tf.keras.metrics.RootMeanSquaredError()])
    model.summary()
    
    model.fit(train_dataset, epochs=20, verbose=0)
    
    train_loss, train_accuracy = model.evaluate(train_dataset, verbose=0)
    val_loss, val_accuracy = model.evaluate(val_dataset, verbose=0)
    
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

print('Average training loss:', np.mean(train_losses))
print('Average training accuracy:', np.mean(train_accuracies))
print('Training losses:', train_losses)
print('Training accuracies:', train_accuracies)
print('Average validation loss:', np.mean(val_losses))
print('Average validation accuracy:', np.mean(val_accuracies))
print('Validation losses:', val_losses)
print('Validation accuracies:', val_accuracies)